In [0]:
import tensorflow as tf
from tensorflow import keras

from keras.utils import np_utils

from keras.models import Sequential
from keras.layers import Dense, Activation

import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


### 데이터셋 설정

In [0]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

### 데이터 Shape 알아보기

In [0]:
x_train.shape, y_train.shape

((50000, 32, 32, 3), (50000, 1))

In [0]:
x_test.shape, y_test.shape

((10000, 32, 32, 3), (10000, 1))

### 훈련set/검증 set 분리하기

In [0]:
# x_val = x_train[45000:]
# y_val = y_train[45000:]
# x_train = x_train[:45000]
# y_train = y_train[:45000]

### 데이터 전처리/one-hot encoding

In [0]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
# y_val = np_utils.to_categorical(y_val)

In [0]:
y_train.shape

(50000, 10)

### 모델구성하기

In [0]:
X = tf.placeholder(tf.float32, [None, 32, 32, 3])
Y = tf.placeholder(tf.float32, [None, 10])

### CNN 계층 구성

In [0]:
W1 = tf.Variable(tf.random_normal([5,5,3,64], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1,1,1,1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
print(L1)

Tensor("MaxPool:0", shape=(?, 16, 16, 64), dtype=float32)


In [0]:
W2 = tf.Variable(tf.random_normal([3,3,64,32], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
print(L2)

Tensor("MaxPool_1:0", shape=(?, 8, 8, 32), dtype=float32)


In [0]:
W3 = tf.Variable(tf.random_normal([3,3,32,32], stddev=0.01))
L3 = tf.nn.conv2d(L2, W3, strides=[1,1,1,1], padding='SAME')
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
print(L3)

Tensor("MaxPool_2:0", shape=(?, 4, 4, 32), dtype=float32)


In [0]:
W4 = tf.Variable(tf.random_normal([4*4*32, 128], stddev=0.01))
L4 = tf.reshape(L3, [-1, 4*4*32])
L4 = tf.matmul(L4, W4)
L4 = tf.nn.relu(L4)

In [0]:
W5 = tf.Variable(tf.random_normal([128,10], stddev=0.01))
model = tf.matmul(L4, W5)

In [0]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer=tf.train.AdamOptimizer(0.001).minimize(cost)

In [0]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size= 100
total_batch = x_train.shape[0]/batch_size
total_batch

500.0

In [0]:
# total_cost = 0
# epoch = 0

# for i in range(total_batch):
      

In [0]:
def next_batch(num, data, labels):
  
  """
  num갯수 만큼의 랜덤한 샘플들과 레이블들을 리턴합니다.
  """
  
  idx = np.arange(0, len(data))
  np.random.shuffle(idx)
  idx = idx[:num]
  data_shuffle = [data[i] for i in idx]
  labels_shuffle = [labels[i] for i in idx]
  
  return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [0]:
for epoch in range(11):
  total_cost = 0
  
  for i in range(int(total_batch)):
    batch_xs, batch_ys= next_batch(10, x_train, y_train)
    
    batch_xs = batch_xs.reshape(-1, 32, 32, 3)
    _, cost_val = sess.run([optimizer, cost],
                          feed_dict = {X: batch_xs, Y:batch_ys})
    
    total_cost += cost_val
    
  print('Epoch:' '{%d}' %(epoch+1), 'Avg. cost =', '{:.2f}'.format(total_cost/total_batch))
print('완료!')

Epoch:{1} Avg. cost = 1.35
Epoch:{2} Avg. cost = 1.33
Epoch:{3} Avg. cost = 1.31
Epoch:{4} Avg. cost = 1.33
Epoch:{5} Avg. cost = 1.31
Epoch:{6} Avg. cost = 1.27
Epoch:{7} Avg. cost = 1.31
Epoch:{8} Avg. cost = 1.25
Epoch:{9} Avg. cost = 1.28
Epoch:{10} Avg. cost = 1.27
Epoch:{11} Avg. cost = 1.28
완료!


In [0]:
# How to Evaluate? 
test_loss, test_acc = model.evaluate(x_test, y_test)


AttributeError: ignored

In [0]:
# How to Evaluate?

correct_prediction = tf.equal(tf.argmax(model(activation), 1), tf.argmax(Y,1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy:", accuracy.eval({X: x_test, Y: y_test}))

ValueError: ignored

In [0]:
    # 추가한 코드. Label과 Prediction이 같은 값을 출력하면 맞는 것이다.
    import random
    r = random.randrange(mnist.test.num_examples)
    print('Label : ', sess.run(tf.argmax(mnist.test.labels[r:r+1], 1)))
    print('Prediction :', sess.run(tf.argmax(activation, 1), {x: mnist.test.images[r:r+1]}))

    # 1줄로 된 것을 28x28로 변환
    import matplotlib.pyplot as plt
    plt.imshow(mnist.test.images[r:r+1].reshape(28, 28), cmap='Greys', interpolation='nearest')
    plt.show()

    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(activation, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))


출처: https://pythonkim.tistory.com/26 [파이쿵]